In [1]:
import requests
import time
import pandas as pd

# Get first wallets

In [2]:
def get_first_miner_wallets(start, end):

    base_url = "https://blockchain.info/block-height/"
    miner_wallets = set()
    current_block = start  # Start from genesis block (block 0)
    
    while len(miner_wallets) < end:
        try:
            # Get block data
            response = requests.get(f"{base_url}{current_block}?format=json")
            response.raise_for_status()
            block_data = response.json()
            
            # Extract coinbase transaction (miner reward)
            for block in block_data['blocks']:
                tx_index = 0  # First transaction is coinbase
                tx_hash = block['tx'][tx_index]['hash']
                
                # Get transaction details
                tx_url = f"https://blockchain.info/rawtx/{tx_hash}"
                tx_response = requests.get(tx_url)
                tx_response.raise_for_status()
                tx_data = tx_response.json()
                
                # Find the output address (miner's wallet)
                for out in tx_data['out']:
                    if 'addr' in out:
                        miner_wallets.add(out['addr'])
                        print(f"Found wallet #{len(miner_wallets)}: {out['addr']} in block {current_block}")
                        if len(miner_wallets) >= end:
                            return list(miner_wallets)[:end]
                
            current_block += 1
            
            # Respect API rate limits (1 request per 10 seconds)
            time.sleep(10)
            
        except Exception as e:
            print(f"Error processing block {current_block}: {str(e)}")
            time.sleep(60)  # Wait longer if there's an error
            continue
    
    return list(miner_wallets)[:end]

In [ ]:
start = 0
end = 3

print("Starting to collect miner reward wallets...")
wallets = get_first_miner_wallets(start, end)


df = pd.DataFrame(wallets, columns=["Wallet Address"])
df.to_csv("miner_wallets_"+str(start+1)+"_"+str(end)+".csv", index=False)

Starting to collect miner reward wallets...
Found wallet #1: 1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa in block 0
Found wallet #2: 12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX in block 1
Found wallet #3: 1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1 in block 2


# Check for wallets with balance

In [5]:
from tqdm import tqdm
tqdm.pandas()  # Enable progress_apply for pandas

def get_btc_balance(wallet_address):
    try:
        response = requests.get(f'https://blockchain.info/balance?active={wallet_address}')
        data = response.json()
        return data[wallet_address]['final_balance'] / 100000000  # Convert from satoshi to BTC
    except Exception as e:
        print(f"Error checking {wallet_address}: {str(e)}")
        return None

In [6]:
start = 0
end = 3

df = pd.read_csv("miner_wallets_"+str(start+1)+"_"+str(end)+".csv")

df

,Wallet Address
0,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX
2,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1


In [7]:
# Assuming your DataFrame is called df with a 'wallet_address' column
df['balance_btc'] = df['Wallet Address'].progress_apply(get_btc_balance)

100%|██████████| 3/3 [00:01<00:00,  2.65it/s]


In [8]:
df_filtered = df[df['balance_btc'] > 1]
df_filtered

,Wallet Address,balance_btc
0,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa,103.034311
1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX,51.352302
2,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1,50.078789


# Let's have some fun!!!

In [9]:
import os
import hashlib
import ecdsa
import base58
import binascii

def generate_private_key():
    """Generate a random 32-byte private key (hex string)"""
    return binascii.hexlify(os.urandom(32)).decode('utf-8')

def private_key_to_wif(private_key_hex, compressed=True):
    """Convert a private key to Wallet Import Format (WIF)"""
    private_key_bytes = binascii.unhexlify(private_key_hex)
    
    # Add prefix (0x80 for mainnet)
    extended_key = b'\x80' + private_key_bytes
    
    if compressed:
        extended_key += b'\x01'
    
    # Double SHA-256 hash
    first_hash = hashlib.sha256(extended_key).digest()
    second_hash = hashlib.sha256(first_hash).digest()
    
    # Add checksum (first 4 bytes of the second hash)
    final_key = extended_key + second_hash[:4]
    
    # Base58 encode
    wif = base58.b58encode(final_key)
    return wif.decode('utf-8')

def private_key_to_public_key(private_key_hex, compressed=True):
    """Convert private key to public key using ECDSA"""
    private_key_bytes = binascii.unhexlify(private_key_hex)
    
    # Get ECDSA public key
    sk = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1)
    vk = sk.get_verifying_key()
    
    if compressed:
        # Compressed public key: prefix with 02 or 03 depending on y parity
        x = vk.to_string()[:32]
        y = vk.to_string()[32:]
        prefix = b'\x02' if y[-1] % 2 == 0 else b'\x03'
        public_key = prefix + x
    else:
        # Uncompressed public key: prefix with 04
        prefix = b'\x04'
        public_key = prefix + vk.to_string()
    
    return binascii.hexlify(public_key).decode('utf-8')

def public_key_to_address(public_key_hex):
    """Convert public key to Bitcoin address"""
    public_key_bytes = binascii.unhexlify(public_key_hex)
    
    # SHA-256 hash
    sha256_hash = hashlib.sha256(public_key_bytes).digest()
    
    # RIPEMD-160 hash
    ripemd160 = hashlib.new('ripemd160')
    ripemd160.update(sha256_hash)
    ripemd160_hash = ripemd160.digest()
    
    # Add network byte (0x00 for mainnet)
    extended_hash = b'\x00' + ripemd160_hash
    
    # Double SHA-256 hash for checksum
    first_hash = hashlib.sha256(extended_hash).digest()
    second_hash = hashlib.sha256(first_hash).digest()
    
    # Add checksum (first 4 bytes of the second hash)
    final_hash = extended_hash + second_hash[:4]
    
    # Base58 encode
    address = base58.b58encode(final_hash)
    return address.decode('utf-8')

def generate_bitcoin_wallet():
    """Generate a complete Bitcoin wallet (private key, WIF, and addresses)"""
    # Generate private key
    private_key = generate_private_key()
    
    # Get WIF formats
    wif_compressed = private_key_to_wif(private_key, compressed=True)
    wif_uncompressed = private_key_to_wif(private_key, compressed=False)
    
    # Get public keys
    public_key_compressed = private_key_to_public_key(private_key, compressed=True)
    public_key_uncompressed = private_key_to_public_key(private_key, compressed=False)
    
    # Get addresses
    address_compressed = public_key_to_address(public_key_compressed)
    address_uncompressed = public_key_to_address(public_key_uncompressed)
    
    return {
        'private_key_hex': private_key,
        'wif_compressed': wif_compressed,
        'wif_uncompressed': wif_uncompressed,
        'public_key_compressed': public_key_compressed,
        'public_key_uncompressed': public_key_uncompressed,
        'address_compressed': address_compressed,
        'address_uncompressed': address_uncompressed
    }

def print_wallet(wallet):
    """Print wallet information in a readable format"""
    print("Private Key (hex):", wallet['private_key_hex'])
    print("WIF Compressed:", wallet['wif_compressed'])
    print("WIF Uncompressed:", wallet['wif_uncompressed'])
    print("Public Key Compressed:", wallet['public_key_compressed'])
    print("Public Key Uncompressed:", wallet['public_key_uncompressed'])
    print("Address Compressed:", wallet['address_compressed'])
    print("Address Uncompressed:", wallet['address_uncompressed'])
    print("-" * 50)



In [ ]:
for i in range(2):
    print(f"\nWallet #{i+1}:")
    wallet = generate_bitcoin_wallet()
    print_wallet(wallet)


Wallet #1:
Private Key (hex): e20c9d12db80416fcf6d0f6190ec9a7429fe8e390515cef8cfbb33cef7749784
WIF Compressed: L4o7yiwFLeFPBdbMJTfZAXSwLwVDBa8VPNQvyydf3PmoACuinHD9
WIF Uncompressed: 5KXqhwpBVvJJiNtHP5gYkr3FaX1xKi39hQtBsX3sJm8RxcBqJWc
Public Key Compressed: 031aee1bf9ca1b39932db6cb4a52302e3e65cf71a8bd910de4cdc7eab498c22f32
Public Key Uncompressed: 041aee1bf9ca1b39932db6cb4a52302e3e65cf71a8bd910de4cdc7eab498c22f320f055e5f89c6a876d286c8a9b40918d05685f1db58a5471f5472e639839175df
Address Compressed: 16DpsMeVjjJf1QhPnH3c4v4gtkz8rQYRjC
Address Uncompressed: 1FVFUGy56Z67xnyAnzkDi42oWaqUfr4BGz
--------------------------------------------------

Wallet #2:
Private Key (hex): 1231a1933f5cf140296e3fb5af97df28781a87ea88f609193cb660a41c0ee5a5
WIF Compressed: Kwq5SbSstYyoUPtKuQ36BtNJDB9JjmomHroLioakdvjxpMmHrQNU
WIF Uncompressed: 5HxJKVCJRNHwwEo4q82qJRKsa7q6qCxudSnPbPni9pX8DBNgzzN
Public Key Compressed: 0342d191f6d1002d58bdb737bad0978baaec0d93f1d5ca94dc453b81ece77fa057
Public Key Uncompressed: 0442d1

In [16]:
for i in range(2):
    print(f"\nWallet #{i+1}:")
    wallet = generate_bitcoin_wallet()

    address_comp = wallet['address_compressed']
    address_uncomp = wallet['address_uncompressed']

    print('Private key (hex):', wallet['private_key_hex'])
    print(address_uncomp in df_filtered['Wallet Address'])
    print(address_comp in df_filtered['Wallet Address'])


Wallet #1:
Private key (hex): 7bec3626e4f565e74ee51131f372a2e7a971c5dd7cb0566f98b9b62e609979b8
False
False

Wallet #2:
Private key (hex): f7cd443ee1dd70c14cdae285ca5fc9afa9b767e3eaa15a4ca49ca135bf4207f0
False
False
